In [78]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu

In [80]:
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder'  #path to the folder containing the images
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\PastData\helical_size'
first_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\first_half\first_quarter'
second_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\first_half\second_quarter'
third_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half\third_quarter'
fourth_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half\fourth_quarter'


In [81]:
# multi-input dataset
class CustomImageDataset(Dataset):
    def __init__(self, first_dir, second_dir, third_dir, fourth_dir, transform=None):
        self.first_dir = first_dir
        self.second_dir = second_dir
        self.third_dir = third_dir
        self.fourth_dir = fourth_dir
        self.transform = transform
        # self.images1 = sorted(os.listdir(first_dir))
        self.images1 = sorted(os.listdir(first_dir), key=self.custom_sort_key)
        self.images2 = sorted(os.listdir(second_dir), key=self.custom_sort_key)
        self.images3 = sorted(os.listdir(third_dir), key=self.custom_sort_key)
        self.images4 = sorted(os.listdir(fourth_dir), key=self.custom_sort_key)
        self.labels1 = [self.extract_label(img) for img in self.images1]
        self.labels2 = [self.extract_label(img) for img in self.images2]
        self.labels3 = [self.extract_label(img) for img in self.images3]
        self.labels4 = [self.extract_label(img) for img in self.images4]
        
        #input verification
        assert len(self.images1) == len(self.images2) == len(self.images3) == len(self.images4)

        self.length = len(self.images1)

    def __len__(self):
        return self.length
        # return len(self.images1)

    def __getitem__(self, idx):
        img_name1 = os.path.join(self.first_dir, self.images1[idx])
        image1 = Image.open(img_name1)
        img_name2 = os.path.join(self.second_dir, self.images2[idx])
        image2 = Image.open(img_name2)
        img_name3 = os.path.join(self.third_dir, self.images3[idx])
        image3 = Image.open(img_name3)
        img_name4 = os.path.join(self.fourth_dir, self.images4[idx])
        image4 = Image.open(img_name4)
    
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
            image3 = self.transform(image3)
            image4 = self.transform(image4)
        
        # print(self.labels1)
        # print(self.labels2)
        # print(self.labels3)
        # print(self.labels4)

        label1 = self.labels1[idx]
        label2 = self.labels2[idx]
        label3 = self.labels3[idx]
        label4 = self.labels4[idx]
        return image1, image2, image3, image4, label1, label2, label3, label4

    def custom_sort_key(self, item):
        # Extract numbers after 'Fig_' and 't-'
        fig_number = float(item.split('Fig_')[1].split('__')[0])
        t_number = float(item.split('t-')[1].split('_')[0])
        
        return fig_number, t_number
    
    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5]) #this is the right code
        # label = str(img_name)
        return label

In [82]:
data_transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.45), (0.25))]) 

custom_dataset = CustomImageDataset(first_dir=first_dir, second_dir=second_dir, third_dir=third_dir, fourth_dir=fourth_dir, transform=data_transform)

# # Accessing the data
# for img, label in custom_dataset:
#     print(f"Image shape: {img.shape}, Label: {label}")

print(len(custom_dataset))

# train_set, val_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.15), int(len(custom_dataset)*0.100056)]) #splits data into training, validation and test sets
train_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.25013)])
print(len(train_set))
# print(len(val_set))
print(len(test_set))

3950
2962
988


In [160]:
#hyperparameters
num_epochs = 30
batch_size = 1
learning_rate = 0.0005


train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test = DataLoader(test_set, batch_size=batch_size, shuffle=True)
for (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in train:
    print(labels1.item(), labels2.item(), labels3.item(), labels4.item())
    break

10.23119 1.614382 16.8034 9.827634


In [154]:
class ConvNet(nn.Module): # note need to find out image size
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,30,20, padding='same') #in_channels, out_channels, kernel_size
        self.normalise1 = nn.BatchNorm2d(30)
        # self.pool = nn.MaxPool2d(5,5) #kernel_size, stride (shift x pixel to the right)
        self.pool1 = nn.AvgPool2d(10, stride=10)
        # self.pool1 = nn.MaxPool2d(10, stride=10)
        self.conv2 = nn.Conv2d(30, 30, 20, padding='same')
        # self.normalise2 = nn.BatchNorm2d(16)
        # self.pool2 = nn.AvgPool2d(2, stride=2)
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(1, 30, 10, padding='same')
        # self.normalise3 = nn.BatchNorm2d(32) 
        self.conv4 = nn.Conv2d(30, 30, 10, padding='same')
        self.fc0 = nn.Linear(30, 10)
        self.fc = nn.Linear(32*5*5, 1)
        self.fc1 = nn.Linear(60, 40)
        self.fc2 = nn.Linear(40,20)
        self.fc3 = nn.Linear(20,4)
        self.dropout = nn.Dropout(0.1)


    def forward(self, x, y, z, a):
        # print(x.shape)
        x = self.pool1(F.relu(self.normalise1(self.conv1(x)))) 
        x = self.pool1(F.relu(self.normalise1(self.conv2(x)))) 
        # x = x.view(-1, 30)
        x = torch.flatten(x)  #flatten

        x = self.fc0(x)
        y = self.pool1(F.relu(self.normalise1(self.conv1(y)))) 
        y = self.pool1(F.relu(self.normalise1(self.conv2(y)))) 
        # y = y.view(-1, 30)
        y = torch.flatten(y)  #flatten
        y = self.fc0(y)

        z = self.pool1(F.relu(self.normalise1(self.conv1(z))))
        z = self.pool1(F.relu(self.normalise1(self.conv2(z)))) 
        # z = z.view(-1, 30)
        z = torch.flatten(z)  #flatten
        z = self.fc0(z)

        a = self.pool1(F.relu(self.normalise1(self.conv1(a)))) 
        a = self.pool1(F.relu(self.normalise1(self.conv2(a))))
        # a = a.view(-1, 30) 
        a = torch.flatten(a) #flatten
        a = self.fc0(a)
        # print(a.shape)
        # xy = torch.cat((x,y), dim=1)
        # za = torch.cat((z,a), dim=1)
        combined = torch.stack([x,y,z,a])
        combined = combined.unsqueeze(0)
        combined= combined.unsqueeze(0)
        # print(combined.shape)
        # print(combined)
        combined = self.pool2(F.relu(self.normalise1(self.conv3(combined)))) 
        # print(combined.shape)
        # print(combined)
        combined = self.pool2(F.relu(self.normalise1(self.conv4(combined)))) 
        # combined = combined.view(-1, 32*5*5)  #flatten
        combined = torch.flatten(combined)
        combined = F.relu(self.fc1(combined))
        combined = F.relu(self.fc2(combined))
        combined = self.fc3(combined)
        return combined

In [158]:
# model_test = ConvNet().to(device)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, len(param.data))

conv1.weight 30
conv1.bias 30
normalise1.weight 30
normalise1.bias 30
conv2.weight 30
conv2.bias 30
conv3.weight 30
conv3.bias 30
conv4.weight 30
conv4.bias 30
fc0.weight 10
fc0.bias 10
fc.weight 1
fc.bias 1
fc1.weight 40
fc1.bias 40
fc2.weight 20
fc2.bias 20
fc3.weight 4
fc3.bias 4


In [155]:
model = ConvNet().to(device)

# loss and optimizer
criterion = nn.MSELoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
#training loop
n_total_steps = len(train)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in enumerate(train):
        images1 = images1.to(device)
        images2 = images2.to(device)
        images3 = images3.to(device)    
        images4 = images4.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)

        #forward
        outputs = model(images1, images2, images3, images4)
        # print(labels)
        labels1 = labels1.float() 
        labels2 = labels2.float()
        labels3 = labels3.float()
        labels4 = labels4.float()
        labels = torch.cat((labels1, labels2, labels3, labels4), dim=0)
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # if (i+1) % 1000 ==0:
        # print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
        total_loss += loss.item()
    print(f'epoch {epoch+1}/{num_epochs}, average loss = {total_loss/len(train):.4f}')
            


epoch 1/30, average loss = 49.6909
epoch 2/30, average loss = 34.6419
epoch 3/30, average loss = 27.7719
epoch 4/30, average loss = 24.2553
epoch 5/30, average loss = 21.5362
epoch 6/30, average loss = 18.7085
epoch 7/30, average loss = 14.8710
epoch 8/30, average loss = 13.2702
epoch 9/30, average loss = 12.0546
epoch 10/30, average loss = 10.7654
epoch 11/30, average loss = 10.8382
epoch 12/30, average loss = 9.6679
epoch 13/30, average loss = 9.1287
epoch 14/30, average loss = 8.8848
epoch 15/30, average loss = 8.7204
epoch 16/30, average loss = 8.1915
epoch 17/30, average loss = 7.8119
epoch 18/30, average loss = 7.3344
epoch 19/30, average loss = 7.2403
epoch 20/30, average loss = 7.1078
epoch 21/30, average loss = 6.5966
epoch 22/30, average loss = 6.4736
epoch 23/30, average loss = 6.6457
epoch 24/30, average loss = 6.3162
epoch 25/30, average loss = 5.8395
epoch 26/30, average loss = 6.1771
epoch 27/30, average loss = 6.0876
epoch 28/30, average loss = 5.7872
epoch 29/30, avera

In [168]:
#Testing
with torch.no_grad(): # no need to calculate gradient
    squared_difference = 0
    for (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in test:
        images1 = images1.to(device)
        images2 = images2.to(device)
        images3 = images3.to(device)    
        images4 = images4.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)

        #forward
        outputs = model(images1, images2, images3, images4)
    
        # print(predictions)
        squared_difference += (float(outputs[0]) - labels1) ** 2
        squared_difference += (float(outputs[1]) - labels2) ** 2
        squared_difference += (float(outputs[2]) - labels3) ** 2
        squared_difference += (float(outputs[3]) - labels4) ** 2
    
    rmse = torch.sqrt(squared_difference / len(test))
    print(f'RMSE = {rmse}')

tensor([8.5171, 8.8292, 7.8743, 7.0947])
8.51711654663086
8.829228401184082
tensor([21.8942, 10.7292, 15.1234, 18.3342])
21.894227981567383
10.729226112365723
tensor([7.6185, 7.3186, 6.5615, 6.3843])
7.618494510650635
7.318636894226074
tensor([11.5266, 12.0496, 10.7975,  8.9831])
11.526570320129395
12.04959487915039
tensor([19.1744, 15.2440, 13.2020, 11.4082])
19.174423217773438
15.243998527526855
tensor([19.3823,  8.9178,  9.8667,  9.8582])
19.3823184967041
8.917823791503906
tensor([21.3412, 20.1608, 14.4017, 12.8504])
21.341230392456055
20.16084098815918
tensor([13.6273, 16.9631,  6.0901, 12.2402])
13.627279281616211
16.96314811706543
tensor([15.3094, 12.5132, 11.7066, 10.4645])
15.309435844421387
12.513203620910645
tensor([18.2371, 22.2905, 10.0835,  9.7225])
18.237058639526367
22.290456771850586
tensor([2.0394, 2.1253, 4.1774, 2.7238])
2.039350986480713
2.125279664993286
tensor([5.3836, 6.3289, 6.7027, 6.8675])
5.383626937866211
6.32893180847168
tensor([ 9.0851, 10.9396, 10.6869,  

In [ ]:
#archived 
# class ConvNet(nn.Module): # note need to find out image size
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         self.conv1 = nn.Conv2d(1,30,20, padding='same') #in_channels, out_channels, kernel_size
#         self.normalise1 = nn.BatchNorm2d(30)
#         # self.pool = nn.MaxPool2d(5,5) #kernel_size, stride (shift x pixel to the right)
#         self.pool1 = nn.AvgPool2d(10, stride=10)
#         # self.pool1 = nn.MaxPool2d(10, stride=10)
#         self.conv2 = nn.Conv2d(30, 30, 20, padding='same')
#         # self.normalise2 = nn.BatchNorm2d(16)
#         # self.pool2 = nn.AvgPool2d(2, stride=2)
#         self.pool2 = nn.MaxPool2d(2, stride=2)
#         self.conv3 = nn.Conv2d(1, 30, 10, padding='same')
#         # self.normalise3 = nn.BatchNorm2d(32) 
#         self.conv4 = nn.Conv2d(30, 30, 10, padding='same')
#         self.fc0 = nn.Linear(30, 10)
#         self.fc = nn.Linear(32*5*5, 1)
#         self.fc1 = nn.Linear(60, 40)
#         self.fc2 = nn.Linear(40,20)
#         self.fc3 = nn.Linear(20,4)
#         self.dropout = nn.Dropout(0.1)